<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de los discos: Sed, Presión, Rocanroles sin destino y Señales, de la banda argentina Callejeros.

In [2]:
df = pd.read_csv('callejeros.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,Tener que seguir
1,Tener que alimentar
2,Sin correr a chetearla
3,Siguen dando vueltas y poniendo
4,Y nunca sacan sortija


In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1564


### 1 - Preprocesamiento

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [5]:
# Demos un vistazo
sentence_tokens[:2]

[['tener', 'que', 'seguir'], ['tener', 'que', 'alimentar']]

### 2 - Crear los vectores (word2vec)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [7]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=200,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [8]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [9]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1564


In [10]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 306


### 3 - Entrenar el modelo generador

In [11]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 68310.1171875
Loss after epoch 1: 29678.0703125
Loss after epoch 2: 27518.8671875
Loss after epoch 3: 26907.3984375
Loss after epoch 4: 26963.125
Loss after epoch 5: 26807.09375
Loss after epoch 6: 26076.078125
Loss after epoch 7: 26833.78125
Loss after epoch 8: 26929.6875
Loss after epoch 9: 25992.5625
Loss after epoch 10: 25149.59375
Loss after epoch 11: 26530.9375
Loss after epoch 12: 25994.9375
Loss after epoch 13: 25800.5625
Loss after epoch 14: 25732.625
Loss after epoch 15: 25964.5
Loss after epoch 16: 25343.6875
Loss after epoch 17: 25560.53125
Loss after epoch 18: 25299.90625
Loss after epoch 19: 25111.5625
Loss after epoch 20: 24277.875
Loss after epoch 21: 23602.875
Loss after epoch 22: 23690.25
Loss after epoch 23: 22630.0
Loss after epoch 24: 22581.6875
Loss after epoch 25: 22673.8125
Loss after epoch 26: 22449.25
Loss after epoch 27: 22056.375
Loss after epoch 28: 21224.25
Loss after epoch 29: 21596.1875
Loss after epoch 30: 20852.5625
Loss after epoch

(404767, 1026800)

### 4 - Ensayar

In [12]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["razón"], topn=10)

[('aunque', 0.5738430619239807),
 ('luz', 0.5717668533325195),
 ('voz', 0.5605605840682983),
 ('parar', 0.5370591878890991),
 ('alegría', 0.5337885618209839),
 ('locura', 0.5311153531074524),
 ('sin', 0.49835002422332764),
 ('escapar', 0.488768070936203),
 ('dirección', 0.47467589378356934),
 ('noche', 0.46884673833847046)]

In [13]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["ilusión"], topn=10)

[('pena', -0.1148681789636612),
 ('volver', -0.15355628728866577),
 ('día', -0.1578645259141922),
 ('corazón', -0.16512562334537506),
 ('cosas', -0.16604292392730713),
 ('sin', -0.16677871346473694),
 ('yo', -0.17177680134773254),
 ('pero', -0.17791247367858887),
 ('por', -0.1835840344429016),
 ('cuando', -0.18358924984931946)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ilusión"], topn=10)

[('adicción', 0.7125499844551086),
 ('gente', 0.6944039463996887),
 ('giles', 0.6940250396728516),
 ('ayuda', 0.6618411540985107),
 ('único', 0.6486877202987671),
 ('soledad', 0.6450608372688293),
 ('alma', 0.6019098162651062),
 ('angustia', 0.5956469774246216),
 ('quedar', 0.5920230150222778),
 ('dos', 0.5883969664573669)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rocanrol"], topn=5)

[('eh', 0.8393474817276001),
 ('tocar', 0.7185860872268677),
 ('siento', 0.6735626459121704),
 ('este', 0.6685516238212585),
 ('recuerdo', 0.6515269875526428)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["silencio"])

[('escapar', 0.6860098838806152),
 ('infierno', 0.6114853620529175),
 ('duro', 0.6015012860298157),
 ('lleno', 0.569878339767456),
 ('mundo', 0.5604426264762878),
 ('amargo', 0.5586040019989014),
 ('desde', 0.5550762414932251),
 ('fiel', 0.5469464063644409),
 ('nada', 0.5190041661262512),
 ('cemento', 0.515375018119812)]

In [17]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(positive=["emoción"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [19]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.

# Conclusiones

Si bien podemos encontrar relaciones entre palabras que son acordes al estilo de letras de la banda, es posible notar que en el espacio generado por la proyección del TSNE, las palabras se encuentran bastante dispersas, y no es posible encontrar clusters significativos. Quizás esto se deba a que el conjunto de datos no corresponde con la discografía completa de la banda y a que suelen utilizar las mismas palabras pero en diversos contextos.